### Note
This notebook was created to be run in Google Colab.

In [ ]:
# Get GRAM data
from google.colab import drive
drive.mount('/content/gdrive')
!cp '/content/gdrive/My Drive/GRAM.tar.xz' .
!tar --extract --xz -f GRAM.tar.xz
drive.flush_and_unmount()
!rm GRAM.tar.xz

Mounted at /content/gdrive


In [ ]:
# Clone code repo
!git clone https://github.com/RodrigoDLPontes/cs7643-final-project.git
!mv cs7643-final-project/* .
!rm -rf cs7643-final-project

Cloning into 'cs7643-final-project'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 38 (delta 12), reused 36 (delta 10), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [ ]:
# Create GRAM PyTorch Dataset object
from gram import GRAM_RTM
gram_dataset = GRAM_RTM()

In [ ]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
NUM_CLASSES = 15 # Max number of cars found in a single picture in training data
NUM_FEATURES = 64 # Don't change cuz this the default value


In [ ]:
class MyModuleList(nn.ModuleList):
    def __add__(self, x):
        tmp = [m for m in self.modules()] + [m for m in x.modules()]
        return MyModuleList(tmp)
    def forward(self, x):
        for layer in self:
            x = layer(x)
        return x

def make_basic_block(inplanes, planes, stride=1, downsample=None):
    def conv3x3(in_planes, out_planes, stride=1):
        return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                         padding=1, bias=False)

    block_list = MyModuleList([
            conv3x3(inplanes, planes, stride),
            nn.BatchNorm2d(planes),
            nn.ReLU(inplace=True),
            conv3x3(planes, planes),
            nn.BatchNorm2d(planes),
    ])
    if downsample == None:
        residual = MyModuleList([])
    else:
        residual = downsample
    return MyModuleList([block_list, residual])

#Specialized NN
class PytorchResNet(nn.Module):
    def __init__(self, section_reps,
                 num_classes=NUM_CLASSES, nbf=NUM_FEATURES,
                 conv1_size=7, conv1_pad=3,
                 downsample_start=True):
        super(PytorchResNet, self).__init__()

        # Since use_basic_block == True
        self.expansion = 1
        self.block_fn = make_basic_block

        self.downsample_start = downsample_start
        self.inplanes = nbf

        self.conv1 = nn.Conv2d(3, nbf, kernel_size=conv1_size,
                               stride=downsample_start + 1, padding=conv1_pad, bias=False)
        self.bn1 = nn.BatchNorm2d(nbf)

        sections = []
        for i, section_rep in enumerate(section_reps):
            sec = self._make_section(nbf * (2 ** i), section_rep, stride=(i != 0) + 1)
            sections.append(sec)
        self.sections = MyModuleList(sections)
        lin_inp = nbf * int(2 ** (len(section_reps) - 1)) * self.expansion \
            if len(self.sections) != 0 else nbf
        self.fc = nn.Linear(lin_inp, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_section(self, planes, num_blocks, stride=1):
        if stride != 1 or self.inplanes != planes * self.expansion:
            downsample = MyModuleList([
                    nn.Conv2d(self.inplanes, planes * self.expansion,
                              kernel_size=1, stride=stride, bias=False),
                    nn.BatchNorm2d(planes * self.expansion),
            ])
        else:
            downsample = None

        blocks = []
        blocks.append(self.block_fn(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * self.expansion
        for i in range(1, num_blocks):
            blocks.append(self.block_fn(self.inplanes, planes))

        return MyModuleList(blocks)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        if self.downsample_start:
            x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)

        for sec_ind, section in enumerate(self.sections):
            for block_ind, (block, shortcut) in enumerate(section):
                x_input = x
                if len(shortcut) != 0:
                    x = shortcut(x)
                x_conv = block(x_input)
                x = x + x_conv
                x = F.relu(x)

        x = F.avg_pool2d(x, (x.size(2), x.size(3)))
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x



In [ ]:
base_model = PytorchResNet([1,1,1,1], num_classes=7, conv1_size=3, conv1_pad=1, nbf=16, downsample_start=False)

In [ ]:
print(base_model)

PytorchResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sections): MyModuleList(
    (0): MyModuleList(
      (0): MyModuleList(
        (0): MyModuleList(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): MyModuleList()
      )
    )
    (1): MyModuleList(
      (0): MyModuleList(
        (0): MyModuleList(
          (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_

In [ ]:
from torch.utils.data import DataLoader
gram_dataset = GRAM_RTM(spec_nn=True)
# Random split of data into training and validation
# Datasets
train_dataset = GRAM_RTM(spec_nn=True, split='train')
val_dataset = GRAM_RTM(spec_nn=True, split='val')
test_dataset = GRAM_RTM(spec_nn=True, split='test')
# DataLoaders
train_loader = torch.utils.data.DataLoader(train_dataset,
                 batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset,
                 batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                 batch_size=16, shuffle=True)
# loader = DataLoader(gram_dataset, batch_size=16, shuffle=True)

In [ ]:
import torch.nn.functional as F
import torch.optim as optim

rcnn = base_model.cuda()
sgd = optim.SGD(rcnn.parameters(), lr=0.1, momentum=0.9)
criterion = F.cross_entropy

def train(epoch, losses):
    '''
    Train the model for one epoch.
    '''
    for batch_idx, batch in enumerate(train_loader):
        images, targets = batch['image'].cuda(), batch['num_cars'].cuda()
        sgd.zero_grad()
        output = rcnn(images)
        loss = criterion(output, targets)
        np_output = output[0].detach().cpu().numpy()
        print(f'\r{epoch}: {loss.item():.2f}', end='')
        losses.append(loss.item())
        loss.backward()
        sgd.step()

In [ ]:
def evaluate(epoch, split):
    '''
    Compute loss on val or test data.
    '''
    loss = 0
    correct = 0
    n_examples = 0
    with torch.no_grad():
        if split == 'val':
            loader = val_loader
        elif split == 'test':
            loader = test_loader
        for batch_idx, batch in enumerate(loader):
            act_maps, targets = batch['image'].cuda(), batch['num_cars'].cuda()
            output = rcnn(act_maps)
            loss += criterion(output, targets).item()
            pred = torch.argmax(output, dim=1)
            if split and batch_idx == 0 == 'test': print(pred)
            correct += (pred == targets).cpu().sum().item()
            n_examples += pred.shape[0]
        loss /= n_examples
        acc = 100. * correct / n_examples
        print(f'{epoch}: {split} set: Average loss: {loss:.4f}, Accuracy: {correct}/{n_examples} ({acc:.0f}%)')
    return loss, acc

In [ ]:
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

train_losses = []
NUM_EPOCHS = 30
for epoch in range(1, NUM_EPOCHS + 1):
    train(epoch, train_losses)
    evaluate(epoch, 'val')


1: 1.061: val set: Average loss: 0.1020, Accuracy: 698/1504 (46%)
2: 0.872: val set: Average loss: 0.1010, Accuracy: 529/1504 (35%)
3: 0.863: val set: Average loss: 0.1028, Accuracy: 692/1504 (46%)
4: 0.924: val set: Average loss: 0.1038, Accuracy: 723/1504 (48%)
5: 0.555: val set: Average loss: 0.1066, Accuracy: 649/1504 (43%)
6: 0.496: val set: Average loss: 0.1028, Accuracy: 784/1504 (52%)
7: 0.207: val set: Average loss: 0.1138, Accuracy: 839/1504 (56%)
8: 0.638: val set: Average loss: 0.1061, Accuracy: 835/1504 (56%)
9: 0.719: val set: Average loss: 0.1241, Accuracy: 677/1504 (45%)
10: 0.4510: val set: Average loss: 0.1394, Accuracy: 783/1504 (52%)
11: 0.3311: val set: Average loss: 0.1236, Accuracy: 919/1504 (61%)
12: 0.4112: val set: Average loss: 0.1682, Accuracy: 717/1504 (48%)
13: 0.0313: val set: Average loss: 0.1568, Accuracy: 813/1504 (54%)
14: 0.0214: val set: Average loss: 0.1274, Accuracy: 847/1504 (56%)
15: 0.0715: val set: Average loss: 0.1515, Accuracy: 714/1504 (47%

TypeError: ignored

In [ ]:
evaluate(epoch=30, split='test')

30: test set: Average loss: 0.2690, Accuracy: 724/1504 (48%)


(0.2689679332394549, 48.138297872340424)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(train_losses)

NameError: ignored